In [1]:
# pandas의 활용이 가능한 경우에는 가급적 pandas를 연계하여 사용
import openpyxl
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from pathlib import Path
import random
import numpy as np
import win32com.client

In [2]:
p1 = Path.cwd() / 'attachments'
p2 = Path.cwd() / 'result_attachments'

In [3]:
# load_workbook으로 불러오기
# gspread의 경우 ss = gc.open('') 형태로 활용
wb = openpyxl.load_workbook(p1 / 'example.xlsx')
# sheetnames 메서드를 통해 시트들 확인 → 단순 문자열에 대한 리스트로 반환
# gspread의 경우 ss.worksheets() 형태로 활용 → worksheet 객체에 대한 리스트로 반환
wb.sheetnames

['Sheet1', 'Sheet2', 'Sheet3']

In [4]:
# active 메서드를 통해 활성 시트를 객체로 지정 가능
# 혹은 sheet = wb['Sheet1'] 형태로도 지정 가능
# gspread의 경우 ss.get_worksheet(0) 혹은 ss.worksheet('') 형태로 활용
sheet = wb['Sheet1']
# 현재 sheet의 제목 확인 가능
sheet.title

'Sheet1'